In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']= '0'

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

In [3]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', 
                                  trust_remote_code = True, torch_dtype = torch.float16,
                                 use_flash_attention_2=True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
_ = model.cuda()

In [5]:
!mkdir iium-confession

In [6]:
import json
import os
from tqdm import tqdm

with open('iium-confession.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        filename = os.path.join('iium-confession', f'{no}.json')
        if os.path.exists(filename):
            continue
        try:
            l = json.loads(l)
            padded = tokenizer([l], return_tensors = 'pt', max_length = 20000).to('cuda')
            v = model.encode(padded).cpu().detach().numpy()[0].tolist()
            data = {
                'text': l,
                'v': v,
            }
            with open(filename, 'w') as f:
                json.dump(data, f)
        except:
            pass

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
25526it [22:08, 19.59it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

